In [1]:
import os
import pandas as pd

SCORE_DIR = "data/scores"

In [2]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]
box_scores

['data/scores/202211070MIA.html',
 'data/scores/202212140DAL.html',
 'data/scores/202212070TOR.html',
 'data/scores/202311110ATL.html',
 'data/scores/202212090UTA.html',
 'data/scores/202210290BRK.html',
 'data/scores/202310280CLE.html',
 'data/scores/202211070DET.html',
 'data/scores/202305060MIA.html',
 'data/scores/202212060DEN.html',
 'data/scores/202211190POR.html',
 'data/scores/202212070MEM.html',
 'data/scores/202211140GSW.html',
 'data/scores/202302260GSW.html',
 'data/scores/202210290CHI.html',
 'data/scores/202304260MIL.html',
 'data/scores/202212140ORL.html',
 'data/scores/202304090NYK.html',
 'data/scores/202301040PHI.html',
 'data/scores/202311080MEM.html',
 'data/scores/202211200BRK.html',
 'data/scores/202303010CHO.html',
 'data/scores/202311080DEN.html',
 'data/scores/202302130CHI.html',
 'data/scores/202210250WAS.html',
 'data/scores/202210260MIL.html',
 'data/scores/202212040SAC.html',
 'data/scores/202211210CHI.html',
 'data/scores/202212070CHI.html',
 'data/scores/

In [3]:
from bs4 import BeautifulSoup

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [4]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [5]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [6]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [7]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 1452
200 / 1452
300 / 1452
400 / 1452
500 / 1452
600 / 1452
700 / 1452
800 / 1452
900 / 1452
1000 / 1452
1100 / 1452
1200 / 1452
1300 / 1452
1400 / 1452


In [9]:
games_df = pd.concat(games, ignore_index=True)
games_df.shape

(2904, 150)

In [13]:
#games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,240.0,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
17768,240.0,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
17769,240.0,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
17770,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [10]:
games_df.to_csv("nba_games_2023_2024.csv")

In [11]:
other_games = pd.read_csv("nba_games_2022.csv")
games = pd.read_csv("nba_games_2023_2024.csv")

other_games.drop(["Unnamed: 0","mp.1"],axis=1,inplace=True)
games.drop(["Unnamed: 0","mp.1"],axis=1,inplace=True)

print(games.columns)
print(other_games.columns)

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'tov%_max_opp', 'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp',
       'team_opp', 'total_opp', 'home_opp', 'season', 'date', 'won'],
      dtype='object', length=149)
Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'tov%_max_opp', 'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp',
       'team_opp', 'total_opp', 'home_opp', 'season', 'date', 'won'],
      dtype='object', length=149)


In [12]:
all_games = pd.concat([other_games, games], ignore_index=True)
all_games.shape

(20676, 149)

In [67]:
all_games.sort_values("date").tail(100)

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
20270,240.0,41.0,94.0,0.436,13.0,38.0,0.342,21.0,26.0,0.808,...,23.1,40.5,205.0,128.0,NOP,125,1,2024,2023-11-02,False
19232,240.0,51.0,98.0,0.520,18.0,37.0,0.486,12.0,13.0,0.923,...,25.4,34.4,139.0,139.0,PHO,121,1,2024,2023-11-02,True
19233,240.0,43.0,80.0,0.538,16.0,32.0,0.500,19.0,26.0,0.731,...,41.0,38.3,159.0,127.0,SAS,132,0,2024,2023-11-02,False
20096,240.0,39.0,85.0,0.459,10.0,29.0,0.345,27.0,36.0,0.750,...,33.3,32.3,231.0,119.0,UTA,113,1,2024,2023-11-02,True
20206,240.0,38.0,79.0,0.481,10.0,28.0,0.357,13.0,19.0,0.684,...,100.0,37.4,169.0,112.0,PHI,114,1,2024,2023-11-02,False
20207,240.0,42.0,92.0,0.457,9.0,25.0,0.360,21.0,25.0,0.840,...,33.3,24.9,200.0,124.0,TOR,99,0,2024,2023-11-02,True
20097,240.0,41.0,89.0,0.461,12.0,33.0,0.364,19.0,26.0,0.731,...,25.5,29.2,155.0,118.0,ORL,115,0,2024,2023-11-02,False
20271,240.0,39.0,80.0,0.488,14.0,32.0,0.438,33.0,35.0,0.943,...,20.5,64.1,187.0,135.0,DET,116,0,2024,2023-11-02,True
18093,240.0,50.0,83.0,0.602,15.0,29.0,0.517,24.0,30.0,0.800,...,41.0,34.6,175.0,138.0,GSW,141,0,2024,2023-11-03,False
20552,240.0,46.0,81.0,0.568,13.0,28.0,0.464,9.0,17.0,0.529,...,33.3,30.3,187.0,118.0,MIA,121,1,2024,2023-11-03,False


In [13]:
all_games.to_csv("nba_games.csv")

In [62]:
test = all_games
test.iloc[::2, :].head()

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,0.778,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
2,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,0.739,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
4,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,0.750,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False
6,240.0,35.0,83.0,0.422,9.0,24.0,0.375,12.0,18.0,0.667,...,33.3,30.9,171.0,105.0,MIN,107,1,2017,2017-03-08,False
8,240.0,44.0,87.0,0.506,10.0,23.0,0.435,27.0,32.0,0.844,...,22.5,38.2,179.0,140.0,DAL,121,1,2019,2019-03-26,True


In [64]:
for col in all_games.columns:
    print(col)

mp
fg
fga
fg%
3p
3pa
3p%
ft
fta
ft%
orb
drb
trb
ast
stl
blk
tov
pf
pts
+/-
ts%
efg%
3par
ftr
orb%
drb%
trb%
ast%
stl%
blk%
tov%
usg%
ortg
drtg
mp_max
mp_max.1
fg_max
fga_max
fg%_max
3p_max
3pa_max
3p%_max
ft_max
fta_max
ft%_max
orb_max
drb_max
trb_max
ast_max
stl_max
blk_max
tov_max
pf_max
pts_max
+/-_max
ts%_max
efg%_max
3par_max
ftr_max
orb%_max
drb%_max
trb%_max
ast%_max
stl%_max
blk%_max
tov%_max
usg%_max
ortg_max
drtg_max
team
total
home
index_opp
mp_opp
mp_opp.1
fg_opp
fga_opp
fg%_opp
3p_opp
3pa_opp
3p%_opp
ft_opp
fta_opp
ft%_opp
orb_opp
drb_opp
trb_opp
ast_opp
stl_opp
blk_opp
tov_opp
pf_opp
pts_opp
+/-_opp
ts%_opp
efg%_opp
3par_opp
ftr_opp
orb%_opp
drb%_opp
trb%_opp
ast%_opp
stl%_opp
blk%_opp
tov%_opp
usg%_opp
ortg_opp
drtg_opp
mp_max_opp
mp_max_opp.1
fg_max_opp
fga_max_opp
fg%_max_opp
3p_max_opp
3pa_max_opp
3p%_max_opp
ft_max_opp
fta_max_opp
ft%_max_opp
orb_max_opp
drb_max_opp
trb_max_opp
ast_max_opp
stl_max_opp
blk_max_opp
tov_max_opp
pf_max_opp
pts_max_opp
+/-_max_opp
ts%_max

In [ ]:
all_games.drop([])